In [1]:
import pandas as pd
import ast
import matplotlib.pyplot as plt


# Load the Excel file
df = pd.read_excel("3d_analysis.xlsx")

# function to convert string representations of lists to actual lists
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        print(f"Skipping invalid entry: {val}")
        return None  # or you could handle this case differently

# Apply the  function to convert strings to lists, handling NaN values for tags_main
df['tags_main'] = df['tags_main'].apply(lambda x: safe_literal_eval(x) if isinstance(x, str) else x)

# Drop rows where 'tags_main' is None
df = df.dropna(subset=['tags_main'])

# Explode the 'tags_main' column so that each element in the lists becomes its own row
df_exploded_main = df.explode('tags_main')

# Count the occurrences of each tag using the value_counts() method
tag_counts_main = df_exploded_main['tags_main'].value_counts()

# Display the counts of each tag
print(tag_counts_main.head(20))

Skipping invalid entry:  as vertically-printed axles won't be as strong as horizontally-printed axles.vertical_axle_fillet_depth_adj
tags_main
3D Models                     1918
3D Printers                    409
Toys & Games                   383
Hobby & Makers                 322
Household                      287
Art & Design                   180
Other Toys & Games             131
Accessories                    128
Action Figures & Statues       127
Gadgets                        118
Organizers                     114
Test Models                    113
Tools                          111
Sculptures                     108
Home Decor                      74
Bambu Lab Parts & Upgrades      66
Seasonal designs                60
Office                          55
World & Scans                   53
Other Art & Designs             50
Name: count, dtype: int64


In [2]:
# Apply the safe_literal_eval function to convert strings to lists, handling NaN values for tags_user
df['tags_user'] = df['tags_user'].apply(lambda x: safe_literal_eval(x) if isinstance(x, str) else x)

# Drop rows where 'tags_user' is None
df = df.dropna(subset=['tags_user'])

# Explode the 'tags_user' column so that each element in the lists becomes its own row
df_exploded_user = df.explode('tags_user')

# Count the occurrences of each tag using the value_counts() method
tag_counts_user = df_exploded_user['tags_user'].value_counts()

# Display the counts of each tag
print("Counts of each tag in 'tags_user':")
print(tag_counts_user.head(20))

Counts of each tag in 'tags_user':
tags_user
toy             125
organizer        87
thingiverse      87
holder           82
printinplace     75
articulated      74
box              69
decoration       64
dragon           58
fidget           54
vase             53
test             49
flexible         49
tool             49
filament         48
starwars         45
desk             40
christmas        40
prusa            40
cute             39
Name: count, dtype: int64


In [4]:
# Ensure no NaN values in 'downloads' column and convert to numeric
df['downloads'] = pd.to_numeric(df['downloads'], errors='coerce').fillna(0)

# Ensure that downloads are correctly propagated after explosion
df_exploded_main['downloads'] = df['downloads'].repeat(df['tags_main'].str.len()).reset_index(drop=True)

# Group by the exploded tags_main and sum the downloads
downloads_by_tag_main = df_exploded_main.groupby('tags_main')['downloads'].sum()

# Sort the results by the sum of downloads in descending order
downloads_by_tag_main_sorted = downloads_by_tag_main.sort_values(ascending=False)

# Display the sorted results for tags_main
print("Sum of downloads for each category in tags_main:")
print(downloads_by_tag_main_sorted.head(20))


Sum of downloads for each category in tags_main:
tags_main
3D Models                     6137724.0
Toys & Games                  1299561.0
3D Printers                   1244585.0
Hobby & Makers                 990398.0
Household                      910243.0
Art & Design                   556995.0
Action Figures & Statues       458275.0
Gadgets                        421630.0
Accessories                    372451.0
Other Toys & Games             369510.0
Sculptures                     347815.0
Organizers                     325185.0
Test Models                    298254.0
Tools                          282939.0
Home Decor                     233952.0
Vehicles                       218220.0
Bambu Lab Parts & Upgrades     217285.0
Seasonal designs               200159.0
Office                         185330.0
World & Scans                  173628.0
Name: downloads, dtype: float64


In [5]:
# Filter the DataFrame to keep only rows where the third element in 'tags_main' exists
df_filtered_main = df[df['tags_main'].apply(lambda x: len(x) > 2 if isinstance(x, list) else False)]

# Extract the third element from 'tags_main'
df_filtered_main['third_tag'] = df_filtered_main['tags_main'].apply(lambda x: x[2])

# Group by the third element and sum the downloads
downloads_by_third_tag = df_filtered_main.groupby('third_tag')['downloads'].sum()

# Sort the results by the sum of downloads in descending order
downloads_by_third_tag_sorted = downloads_by_third_tag.sort_values(ascending=False)

# Display the sorted results for third categories in tags_main
print("Sum of downloads for each third category in tags_main:")
print(downloads_by_third_tag_sorted.head(20))

Sum of downloads for each third category in tags_main:
third_tag
Action Figures & Statues           529924.0
Accessories                        518183.0
Organizers                         471145.0
Tools                              457356.0
Test Models                        439066.0
Other Toys & Games                 429476.0
Sculptures                         348663.0
Home Decor                         288233.0
Bambu Lab Parts & Upgrades         244921.0
Office                             234691.0
Vehicles                           214845.0
Other Art & Designs                197877.0
Kitchen                            178506.0
Outdoor & Garden                   164606.0
Other Gadgets                      149559.0
Other House Equipment              144158.0
Creality Parts & Upgrades          142042.0
Winter & Christmas & New Year's    138799.0
Animals                            132907.0
Board Games                        130700.0
Name: downloads, dtype: float64


In [7]:
# function to convert 'time' from string format to total minutes
def convert_time_to_minutes(time_str):
    if pd.isna(time_str):
        return None
    time_parts = time_str.split()
    total_minutes = 0
    for part in time_parts:
        if 'h' in part:
            try:
                hours = part.replace('h', '')
                if hours.isdigit():
                    total_minutes += int(hours) * 60
                else:
                    print(f"Skipping invalid hour part: {part}")
            except ValueError:
                print(f"Skipping invalid hour part: {part}")
                continue
        elif 'm' in part:
            try:
                minutes = part.replace('m', '')
                if minutes.isdigit():
                    total_minutes += int(minutes)
                else:
                    print(f"Skipping invalid minute part: {part}")
            except ValueError:
                print(f"Skipping invalid minute part: {part}")
                continue
        else:
            print(f"Skipping invalid time format: {part}")
    return total_minutes

# Apply the conversion function to the 'time' column
df['time_minutes'] = df['time'].apply(convert_time_to_minutes)

# Convert minutes to hours
df['time_hours'] = df['time_minutes'] / 60

# Print the new 'time_minutes' and 'time_hours' columns
print(df[['name', 'time', 'time_minutes', 'time_hours']].head(10))

# Show summary statistics using the describe function for minutes
print("\nSummary statistics of the 'time_minutes' column:")
print(df['time_minutes'].describe())

# Show summary statistics using the describe function for hours
print("\nSummary statistics of the 'time_hours' column:")
print(df['time_hours'].describe())


Skipping invalid time format: an
Skipping invalid time format: extra
Skipping invalid time format: 0.1
Skipping invalid minute part: mm
Skipping invalid time format: is
Skipping invalid time format: added
Skipping invalid time format: to
Skipping invalid hour part: this
Skipping invalid time format: setting.
Skipping invalid time format: ¬†Note
Skipping invalid hour part: that
Skipping invalid time format: not
Skipping invalid time format: all
Skipping invalid time format: supports
Skipping invalid time format: obey
Skipping invalid hour part: this
Skipping invalid hour part: setting.technic_pin_center_bore_adj:
Skipping invalid hour part: this
Skipping invalid time format: adjusts
Skipping invalid hour part: the
Skipping invalid minute part: diameter
Skipping invalid time format: of
Skipping invalid hour part: the
Skipping invalid time format: center
Skipping invalid time format: bore
Skipping invalid time format: in
Skipping invalid minute part: most
Skipping invalid hour part: Techn

In [8]:
# Define a function to convert 'time' from string format to total minutes
def convert_time_to_minutes(time_str):
    if pd.isna(time_str):
        return None
    time_parts = time_str.split()
    total_minutes = 0
    for part in time_parts:
        if 'h' in part:
            try:
                hours = part.replace('h', '')
                if hours.isdigit():
                    total_minutes += int(hours) * 60
                else:
                    print(f"Skipping invalid hour part: {part}")
            except ValueError:
                print(f"Skipping invalid hour part: {part}")
                continue
        elif 'm' in part:
            try:
                minutes = part.replace('m', '')
                if minutes.isdigit():
                    total_minutes += int(minutes)
                else:
                    print(f"Skipping invalid minute part: {part}")
            except ValueError:
                print(f"Skipping invalid minute part: {part}")
                continue
        else:
            print(f"Skipping invalid time format: {part}")
    return total_minutes

# Apply the conversion function to the 'time' column
df['time_minutes'] = df['time'].apply(convert_time_to_minutes)

# Convert minutes to hours
df['time_hours'] = df['time_minutes'] / 60

# Sort the models by the longest time to the shortest
df_sorted = df.sort_values(by='time_hours', ascending=False)

# Print the sorted models by longest printing time in hours
print(df_sorted[['name', 'time', 'time_hours']])

# Show summary statistics using the describe function for hours
print("\nSummary statistics of the 'time_hours' column:")
print(df['time_hours'].describe())

Skipping invalid time format: an
Skipping invalid time format: extra
Skipping invalid time format: 0.1
Skipping invalid minute part: mm
Skipping invalid time format: is
Skipping invalid time format: added
Skipping invalid time format: to
Skipping invalid hour part: this
Skipping invalid time format: setting.
Skipping invalid time format: ¬†Note
Skipping invalid hour part: that
Skipping invalid time format: not
Skipping invalid time format: all
Skipping invalid time format: supports
Skipping invalid time format: obey
Skipping invalid hour part: this
Skipping invalid hour part: setting.technic_pin_center_bore_adj:
Skipping invalid hour part: this
Skipping invalid time format: adjusts
Skipping invalid hour part: the
Skipping invalid minute part: diameter
Skipping invalid time format: of
Skipping invalid hour part: the
Skipping invalid time format: center
Skipping invalid time format: bore
Skipping invalid time format: in
Skipping invalid minute part: most
Skipping invalid hour part: Techn